In [42]:
#imports
from datasets import load_dataset
from thai2transformers.metrics import seqeval_classification_metrics
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [49]:
#parameters
class Args:
    dataset_name_or_path = 'thainer'
    feature_col = 'tokens'
    label_col = 'ner_tags'
    metric_for_best_model = 'f1_macro'
    seed = 2020
    data_dir = '~/Downloads/LST20_Corpus'

args = Args()

In [50]:
if args.dataset_name_or_path == 'lst20':
    dataset = load_dataset(args.dataset_name_or_path,data_dir=args.data_dir)
else:
    dataset = load_dataset(args.dataset_name_or_path)
dataset

Dataset thainer downloaded and prepared to /Users/admin/.cache/huggingface/datasets/thainer/thainer/1.3.0/e0a86672e5ad057c1093708597cdda3671a76e9b053d210a32205406726cca92. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 6348
    })
})

In [51]:
if args.dataset_name_or_path == 'thainer' and args.label_col== 'ner_tags':
    dataset = dataset.map(lambda examples: {'ner_tags': [i if i not in [13,26] else 27 for i in examples[args.label_col]]})
    train_valtest_split = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=args.seed)
    dataset['train'] = train_valtest_split['train']
    dataset['validation'] = train_valtest_split['test']
    val_test_split = dataset['validation'].train_test_split(test_size=0.5, shuffle=True, seed=args.seed)
    dataset['validation'] = val_test_split['train']
    dataset['test'] = val_test_split['test']
    tag_labels = dataset['train'].features[args.label_col].feature.names
    tag_labels = [tag_labels[i] for i in range(len(tag_labels)) if i not in [13,26]]
elif args.dataset_name_or_path == 'thainer' and args.label_col== 'pos_tags':
    train_valtest_split = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=args.seed)
    dataset['train'] = train_valtest_split['train']
    dataset['validation'] = train_valtest_split['test']
    val_test_split = dataset['validation'].train_test_split(test_size=0.5, shuffle=True, seed=args.seed)
    dataset['validation'] = val_test_split['train']
    dataset['test'] = val_test_split['test']
    tag_labels = dataset['train'].features[args.label_col].feature.names
else:
    tag_labels = dataset['train'].features[args.label_col].feature.names
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'pos_tags', 'tokens'],
        num_rows: 5078
    })
    validation: Dataset({
        features: ['id', 'ner_tags', 'pos_tags', 'tokens'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'ner_tags', 'pos_tags', 'tokens'],
        num_rows: 635
    })
})

In [52]:
if args.dataset_name_or_path == 'thainer':
    from transformers import AutoTokenizer
    mbert_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    def pre_tokenize(token, space_token):
        token = token.replace(' ', space_token)
        return token
    def is_not_too_long(example,
                        max_length=510):
        tokens = sum([mbert_tokenizer.tokenize(
            pre_tokenize(token, space_token='<_>'))
                      for token in example[args.feature_col]], [])
        return len(tokens) < max_length
    dataset['test'] = dataset['test'].filter(is_not_too_long)
dataset['test']

Dataset({
    features: ['id', 'ner_tags', 'pos_tags', 'tokens'],
    num_rows: 621
})

In [53]:
%%time
#get sentence forms
def generate_sents(dataset, idx):
    features = dataset[idx][args.feature_col]
    labels = dataset[idx][args.label_col]
    return [(features[i], labels[i]) for i in range(len(features))]

train_sents = [generate_sents(dataset['train'],i) for i in range(len(dataset['train']))]
valid_sents = [generate_sents(dataset['validation'],i) for i in range(len(dataset['validation']))]
test_sents = [generate_sents(dataset['test'],i) for i in range(len(dataset['test']))]
len(train_sents), len(valid_sents), len(test_sents)

CPU times: user 1.17 s, sys: 30.8 ms, total: 1.2 s
Wall time: 1.23 s


(5078, 635, 621)

In [54]:
#generate x,y
def extract_features(doc, window=3, max_n_gram=3):
    #padding for words
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]            
    doc_features = []
    
    #for each word
    for i in range(window, len(doc)-window):
        #bias term
        word_features = ['bias'] 
        
        #ngram features
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                
                #word
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
        
        #append to feature per word
        doc_features.append(word_features)
    return doc_features

def generate_xy(all_tuples):
    #target
    y = [[str(l) for (w,l) in t] for t in all_tuples]
    #features
    x_pre = [[w for (w,l) in t] for t in all_tuples]
    x = [extract_features(x_, window=2, max_n_gram = 2) for x_ in tqdm(x_pre)]
    return x, y


x_train, y_train = generate_xy(train_sents)
if args.dataset_name_or_path=='lst20':
    import random
    random.seed(args.seed)
    x_train_small = random.sample(x_train,10000)
    random.seed(args.seed)
    y_train_small =  random.sample(y_train,10000)
else:
    x_train_small = x_train
    y_train_small = y_train
x_valid, y_valid = generate_xy(valid_sents)
x_test, y_test = generate_xy(test_sents)

In [55]:
import pycrfsuite
from sklearn.metrics import classification_report

def train_crf(model_name, c1, c2, x_train, y_train, max_iterations=500):
    # Train model
    trainer = pycrfsuite.Trainer(verbose=True)

    for xseq, yseq in tqdm(zip(x_train, y_train)):
        trainer.append(xseq, yseq)

    trainer.set_params({
        'c1': c1,
        'c2': c2,
        'max_iterations': max_iterations,
        'feature.possible_transitions': True,
        'feature.minfreq': 3.0,
    })

    trainer.train(f'{model_name}_{c1}_{c2}.model')

class Pred:
    def __init__(self,label_ids,predictions):
        self.label_ids = label_ids
        self.predictions = predictions
        
def evaluate_crf(model_path, features, labels, tag_labels):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_path)
    y_pred = []
    for xseq in tqdm(features, total=len(features)): y_pred.append(tagger.tag(xseq))
    idx2tag = {}
    if args.dataset_name_or_path == 'thainer' and args.label_col=='ner_tags':
        idx = [str(i) for i in range(13)] + [str(i) for i in range(14,26)]+ ['27']
    else:
        idx = [str(i) for i in range(len(tag_labels))]
    for i in range(len(idx)):
        idx2tag[idx[i]] = tag_labels[i]
    label_ids = []
    predictions = []
    for s in labels: label_ids.append([idx2tag[i] for i in s])
    for s in y_pred: predictions.append([idx2tag[i] for i in s])
    pred = Pred(label_ids, predictions)
    return seqeval_classification_metrics(pred)

In [122]:
hyperparams = []
for c1 in tqdm([0.,0.5,1.]):
    for c2 in tqdm([0.,0.5,1.]):
        train_crf(args.dataset_name_or_path,c1,c2,x_train_small,y_train_small)
        res = evaluate_crf(f'{args.dataset_name_or_path}_{c1}_{c2}.model',
                                                  x_valid, y_valid, tag_labels)
        res['c1'], res['c2'] = c1, c2
        print(res['classification_report'])
        hyperparams.append(res)


Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 129129
Seconds required: 4.896

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 579570.646983
Feature norm: 5.000000
Error norm: 123054.508418
Active features: 129129
Line search trials: 2
Line search step: 0.000011
Seconds required for this iteration: 3.591

***** Iteration #2 *****
Loss: 395921.452896
Feature norm: 3.511936
Error norm: 92701.019940
Active features: 129129
Line search trials: 3
Line search step: 1.541237
Seconds required for this iteration: 3.957

***** Iteration #3 *****
Loss: 364498.566376
Feature norm: 3.196999
Error norm: 52473.324824
Active features: 129129
Line search trials: 2
Line search step: 0.077037
Seconds req

***** Iteration #44 *****
Loss: 67519.490771
Feature norm: 68.150625
Error norm: 5892.477282
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.061

***** Iteration #45 *****
Loss: 66658.531029
Feature norm: 70.753459
Error norm: 7479.034673
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.028

***** Iteration #46 *****
Loss: 66228.997756
Feature norm: 70.444923
Error norm: 3926.702859
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.077

***** Iteration #47 *****
Loss: 65651.324525
Feature norm: 71.037748
Error norm: 2610.999465
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.047

***** Iteration #48 *****
Loss: 64967.958477
Feature norm: 72.736415
Error norm: 4194.450912
Active features: 129129
Line search trials: 1
Line search step: 1.0

***** Iteration #84 *****
Loss: 44650.249926
Feature norm: 126.025337
Error norm: 1620.965297
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.114

***** Iteration #85 *****
Loss: 44291.844201
Feature norm: 127.126328
Error norm: 3292.597919
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.088

***** Iteration #86 *****
Loss: 43573.867887
Feature norm: 128.088032
Error norm: 1573.621267
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.074

***** Iteration #87 *****
Loss: 43120.702492
Feature norm: 128.622932
Error norm: 1663.027574
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.107

***** Iteration #88 *****
Loss: 42594.095164
Feature norm: 130.151361
Error norm: 5402.022837
Active features: 129129
Line search trials: 1
Line search step

***** Iteration #124 *****
Loss: 29986.109492
Feature norm: 171.056139
Error norm: 1029.633940
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.023

***** Iteration #125 *****
Loss: 29535.829973
Feature norm: 173.683294
Error norm: 1829.151198
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.032

***** Iteration #126 *****
Loss: 29241.047501
Feature norm: 176.239940
Error norm: 1333.483454
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.038

***** Iteration #127 *****
Loss: 29037.260263
Feature norm: 175.682779
Error norm: 685.448287
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.058

***** Iteration #128 *****
Loss: 28876.450028
Feature norm: 175.702001
Error norm: 1123.920839
Active features: 129129
Line search trials: 1
Line search 

***** Iteration #167 *****
Loss: 23428.055339
Feature norm: 215.441955
Error norm: 1644.893700
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.124

***** Iteration #168 *****
Loss: 23331.863512
Feature norm: 218.852349
Error norm: 1134.818874
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.068

***** Iteration #169 *****
Loss: 23242.931216
Feature norm: 218.559569
Error norm: 644.568023
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.088

***** Iteration #170 *****
Loss: 23159.450478
Feature norm: 219.194917
Error norm: 668.350270
Active features: 129129
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.059

***** Iteration #171 *****
Loss: 23065.141289
Feature norm: 220.861415
Error norm: 1144.255433
Active features: 129129
Line search trials: 1
Line search s

...

***** Iteration #338 *****
Loss: 57683.658654
Feature norm: 95.887238
Error norm: 230.128561
Active features: 24150
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.116

***** Iteration #339 *****
Loss: 57683.370175
Feature norm: 95.890114
Error norm: 225.744708
Active features: 24148
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.093

***** Iteration #340 *****
Loss: 57683.069434
Feature norm: 95.885319
Error norm: 235.255210
Active features: 24148
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.091

***** Iteration #341 *****
Loss: 57682.791617
Feature norm: 95.888473
Error norm: 229.132154
Active features: 24148
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.093

***** Iteration #342 *****
Loss: 57682.469990
Feature norm: 95.883694
Error norm: 232.070565
Active features: 24146
Line search trials: 1
Line search step: 1.000000

***** Iteration #384 *****
Loss: 57671.412074
Feature norm: 95.829369
Error norm: 183.020335
Active features: 24084
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.087

***** Iteration #385 *****
Loss: 57671.255878
Feature norm: 95.831084
Error norm: 217.822814
Active features: 24082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.088

***** Iteration #386 *****
Loss: 57670.956640
Feature norm: 95.826079
Error norm: 180.351699
Active features: 24082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.056

***** Iteration #387 *****
Loss: 57670.797046
Feature norm: 95.827642
Error norm: 213.840190
Active features: 24090
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.078

***** Iteration #388 *****
Loss: 57670.507979
Feature norm: 95.822517
Error norm: 182.784619
Active features: 24091
Line search trials: 1
Line search step: 1.000000

***** Iteration #424 *****
Loss: 57662.735426
Feature norm: 95.764053
Error norm: 176.831842
Active features: 24052
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.167

***** Iteration #425 *****
Loss: 57662.465861
Feature norm: 95.765785
Error norm: 139.493976
Active features: 24049
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.223

***** Iteration #426 *****
Loss: 57662.255742
Feature norm: 95.761313
Error norm: 184.497543
Active features: 24048
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.183

***** Iteration #427 *****
Loss: 57661.973383
Feature norm: 95.763225
Error norm: 137.870947
Active features: 24054
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.182

***** Iteration #428 *****
Loss: 57661.756981
Feature norm: 95.758874
Error norm: 178.181341
Active features: 24055
Line search trials: 2
Line search step: 0.500000

***** Iteration #465 *****
Loss: 57654.393481
Feature norm: 95.726521
Error norm: 107.730041
Active features: 24026
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.182

***** Iteration #466 *****
Loss: 57654.225295
Feature norm: 95.723260
Error norm: 113.690931
Active features: 24023
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.201

***** Iteration #467 *****
Loss: 57654.071374
Feature norm: 95.724479
Error norm: 109.882476
Active features: 24024
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.405

***** Iteration #468 *****
Loss: 57653.914362
Feature norm: 95.721193
Error norm: 117.336865
Active features: 24023
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 2.396

***** Iteration #469 *****
Loss: 57653.742145
Feature norm: 95.722351
Error norm: 106.126726
Active features: 24022
Line search trials: 2
Line search step: 0.500000


              precision    recall  f1-score   support

        _BRN     0.1667    0.0222    0.0392        45
        _DES     0.9195    0.6419    0.7560      1815
        _DTM     0.7241    0.6751    0.6987      1948
        _LOC     0.7905    0.5712    0.6632      4254
        _MEA     0.6435    0.5315    0.5821      3097
        _NUM     0.6202    0.5472    0.5814      1292
        _ORG     0.7171    0.4815    0.5762      4544
        _PER     0.8272    0.6824    0.7478      4118
        _TRM     0.4848    0.0988    0.1641       162
        _TTL     0.9789    0.8953    0.9352      2016

   micro avg     0.7744    0.6047    0.6791     23291
   macro avg     0.6872    0.5147    0.5744     23291
weighted avg     0.7712    0.6047    0.6749     23291





In [123]:
hyperparams_df = pd.DataFrame(hyperparams).sort_values('f1_macro',ascending=False).reset_index(drop=True)
best_hyperparams = hyperparams_df.iloc[0,:].to_dict()
hyperparams_df[['c1','c2','f1_micro','f1_macro']]

,c1,c2,f1_micro,f1_macro
0,0.5,0.0,0.717296,0.627721
1,1.0,0.0,0.716445,0.622451
2,0.0,0.0,0.688666,0.615289
3,0.0,0.5,0.703625,0.602803
4,0.5,0.5,0.699979,0.590872
5,0.0,1.0,0.694041,0.586303
6,1.0,0.5,0.692479,0.585022
7,0.5,1.0,0.686875,0.580285
8,1.0,1.0,0.679075,0.574405


In [124]:
print(best_hyperparams['classification_report'])

              precision    recall  f1-score   support

        _BRN     0.3158    0.1333    0.1875        45
        _DES     0.9033    0.8083    0.8532      1815
        _DTM     0.7377    0.6946    0.7155      1948
        _LOC     0.7701    0.6653    0.7138      4254
        _MEA     0.6453    0.5680    0.6042      3097
        _NUM     0.6282    0.5689    0.5971      1292
        _ORG     0.6776    0.5440    0.6035      4544
        _PER     0.8165    0.7756    0.7955      4118
        _TRM     0.4375    0.1728    0.2478       162
        _TTL     0.9686    0.9499    0.9592      2016

   micro avg     0.7632    0.6766    0.7173     23291
   macro avg     0.6901    0.5881    0.6277     23291
weighted avg     0.7574    0.6766    0.7137     23291



In [125]:
#final model
c1, c2 = best_hyperparams['c1'], best_hyperparams['c2']
train_crf(f'{args.dataset_name_or_path}_{args.label_col}_best',c1,c2,x_train,y_train)


Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 588510
Seconds required: 23.289

L-BFGS optimization
c1: 0.500000
c2: 0.000000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 6294769.553331
Feature norm: 1.000000
Error norm: 2955330.355206
Active features: 586206
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 15.132

***** Iteration #2 *****
Loss: 4064922.203746
Feature norm: 5.129274
Error norm: 846213.800774
Active features: 585553
Line search trials: 4
Line search step: 0.125000
Seconds required for this iteration: 31.056

***** Iteration #3 *****
Loss: 3307829.621747
Feature norm: 4.231016
Error norm: 823351.642770
Active features: 562709
Line search trials: 1
Line search step: 1.000000
Se

***** Iteration #44 *****
Loss: 237664.202285
Feature norm: 234.379928
Error norm: 31662.058590
Active features: 265085
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.955

***** Iteration #45 *****
Loss: 226402.334024
Feature norm: 240.837793
Error norm: 9338.432305
Active features: 265800
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.737

***** Iteration #46 *****
Loss: 222464.013006
Feature norm: 246.155359
Error norm: 4215.166487
Active features: 263592
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.672

***** Iteration #47 *****
Loss: 213634.897806
Feature norm: 263.726325
Error norm: 4649.514020
Active features: 249695
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.768

***** Iteration #48 *****
Loss: 212696.472886
Feature norm: 265.315835
Error norm: 33709.332524
Active features: 245462
Line search trials: 3
Line sear

***** Iteration #87 *****
Loss: 160339.256365
Feature norm: 434.229664
Error norm: 3861.836117
Active features: 134033
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.908

***** Iteration #88 *****
Loss: 160133.613139
Feature norm: 434.555307
Error norm: 2958.694421
Active features: 133116
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.947

***** Iteration #89 *****
Loss: 159932.164603
Feature norm: 434.966622
Error norm: 3626.316252
Active features: 131995
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.836

***** Iteration #90 *****
Loss: 159732.533293
Feature norm: 435.271993
Error norm: 3527.054727
Active features: 130871
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.735

***** Iteration #91 *****
Loss: 159545.064757
Feature norm: 435.709170
Error norm: 4035.681303
Active features: 129883
Line search trials: 1
Line search

***** Iteration #132 *****
Loss: 155736.472255
Feature norm: 453.790293
Error norm: 2351.163384
Active features: 115371
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.868

***** Iteration #133 *****
Loss: 155711.500146
Feature norm: 454.177206
Error norm: 4056.474973
Active features: 115241
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.885

***** Iteration #134 *****
Loss: 155649.592646
Feature norm: 454.535460
Error norm: 2334.922076
Active features: 115150
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.928

***** Iteration #135 *****
Loss: 155624.302730
Feature norm: 454.897432
Error norm: 3907.930459
Active features: 115007
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.756

***** Iteration #136 *****
Loss: 155565.471347
Feature norm: 455.246077
Error norm: 2316.364293
Active features: 114910
Line search trials: 1
Line s

***** Iteration #172 *****
Loss: 154187.405613
Feature norm: 464.778848
Error norm: 1855.997149
Active features: 112350
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.631

***** Iteration #173 *****
Loss: 154176.857156
Feature norm: 464.980963
Error norm: 3764.888161
Active features: 112246
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.827

***** Iteration #174 *****
Loss: 154117.377949
Feature norm: 465.172207
Error norm: 1605.337866
Active features: 112228
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.762

***** Iteration #175 *****
Loss: 154108.435123
Feature norm: 465.363741
Error norm: 3618.840242
Active features: 112082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.782

***** Iteration #176 *****
Loss: 154050.481711
Feature norm: 465.553945
Error norm: 1626.544302
Active features: 112017
Line search trials: 1
Line s

...

***** Iteration #334 *****
Loss: 151326.969049
Feature norm: 476.131465
Error norm: 1008.466756
Active features: 100993
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.737

***** Iteration #335 *****
Loss: 151323.870210
Feature norm: 476.172151
Error norm: 1626.637083
Active features: 100905
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.717

***** Iteration #336 *****
Loss: 151311.502354
Feature norm: 476.207468
Error norm: 922.962754
Active features: 100869
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.690

***** Iteration #337 *****
Loss: 151309.104248
Feature norm: 476.245421
Error norm: 1636.227311
Active features: 100799
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.863

***** Iteration #338 *****
Loss: 151296.178825
Feature norm: 476.278675
Error norm: 844.737696
Active features: 100736
Line search trials: 1
Line sea

***** Iteration #379 *****
Loss: 151022.988942
Feature norm: 477.365175
Error norm: 457.671554
Active features: 98813
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 15.596

***** Iteration #380 *****
Loss: 151019.777749
Feature norm: 477.389598
Error norm: 1166.662094
Active features: 98655
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.903

***** Iteration #381 *****
Loss: 151015.588537
Feature norm: 477.426935
Error norm: 1571.080288
Active features: 98604
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.948

***** Iteration #382 *****
Loss: 151005.338396
Feature norm: 477.452632
Error norm: 1098.251072
Active features: 98554
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.983

***** Iteration #383 *****
Loss: 151001.704645
Feature norm: 477.489321
Error norm: 1483.762858
Active features: 98570
Line search trials: 1
Line search

***** Iteration #419 *****
Loss: 150813.487467
Feature norm: 478.309989
Error norm: 417.684495
Active features: 97184
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 15.313

***** Iteration #420 *****
Loss: 150811.622151
Feature norm: 478.331071
Error norm: 1047.877533
Active features: 97066
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.624

***** Iteration #421 *****
Loss: 150808.450531
Feature norm: 478.360953
Error norm: 1389.144716
Active features: 97039
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.676

***** Iteration #422 *****
Loss: 150800.141334
Feature norm: 478.380995
Error norm: 869.689908
Active features: 97019
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.794

***** Iteration #423 *****
Loss: 150798.070474
Feature norm: 478.411026
Error norm: 1280.164981
Active features: 97071
Line search trials: 1
Line search 

***** Iteration #461 *****
Loss: 150643.142138
Feature norm: 479.168536
Error norm: 1013.305266
Active features: 95712
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.616

***** Iteration #462 *****
Loss: 150638.227927
Feature norm: 479.187027
Error norm: 731.211107
Active features: 95679
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.722

***** Iteration #463 *****
Loss: 150636.271741
Feature norm: 479.210290
Error norm: 1061.232958
Active features: 95675
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.724

***** Iteration #464 *****
Loss: 150630.852037
Feature norm: 479.225224
Error norm: 662.939215
Active features: 95616
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.607

***** Iteration #465 *****
Loss: 150629.280618
Feature norm: 479.246285
Error norm: 1076.870517
Active features: 95564
Line search trials: 1
Line search s

Storing the model
Number of active features: 94411 (588510)
Number of active attributes: 63545 (1983731)
Number of active labels: 31 (31)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds required: 0.980



In [56]:
#debug
c1, c2 = 0.5, 0.0

if args.dataset_name_or_path=='lst20' and args.label_col=='ner_tags':
    res = evaluate_crf(f'{args.dataset_name_or_path}_{args.label_col}_best_{c1}_{c2}.model',
                                              x_test, y_test, tag_labels[:-1]) #test set of lst20 does not have E_TTL
    print(res['classification_report'])
else:
    res = evaluate_crf(f'{args.dataset_name_or_path}_{args.label_col}_best_{c1}_{c2}.model',
                                              x_test, y_test, tag_labels)
    print(res['classification_report'])


              precision    recall  f1-score   support

        DATE     0.8758    0.8221    0.8481       163
       EMAIL     1.0000    1.0000    1.0000         1
         LAW     0.9000    0.6000    0.7200        15
         LEN     0.9412    0.8000    0.8649        20
    LOCATION     0.7747    0.6770    0.7226       452
       MONEY     1.0000    0.9138    0.9550        58
ORGANIZATION     0.8550    0.7400    0.7934       550
     PERCENT     0.9375    0.9375    0.9375        16
      PERSON     0.8816    0.7941    0.8356       272
       PHONE     0.7500    0.6000    0.6667        10
        TIME     0.8154    0.6235    0.7067        85
         URL     1.0000    0.8571    0.9231         7
         ZIP     1.0000    0.5000    0.6667         2

   micro avg     0.8458    0.7408    0.7898      1651
   macro avg     0.9024    0.7589    0.8185      1651
weighted avg     0.8450    0.7408    0.7889      1651



In [57]:
pd.DataFrame.from_dict(res,orient='index').transpose()

,accuracy,f1_micro,precision_micro,recall_micro,f1_macro,precision_macro,recall_macro,nb_samples,classification_report
0,0.94819,0.789797,0.845781,0.740763,0.818462,0.902403,0.758858,621,precision recall f1-score ...
